<a href="https://colab.research.google.com/github/alesaccoia/DDM_2425/blob/main/AI_001_Huggingface_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Utilizzare GPT-2 con Huggingface Transformers

In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from IPython.display import display, Markdown

## 1. Caricare il modello e il tokenizer
Per prima cosa, dobbiamo caricare sia il modello che il tokenizer di GPT-2.
Useremo la versione base di GPT-2 (124M parametri).

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

display(Markdown(f"Modello caricato su: {device}"))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Modello caricato su: cpu

In [6]:
def example_1_basic_tokenization():
    """Esempio 1: Tokenizzazione base e dimensione del vocabolario"""

    # Mostra la dimensione del vocabolario
    print(f"Dimensione del vocabolario GPT-2: {len(tokenizer)}")

    # Esempio di tokenizzazione
    text = "Hello, how are you?"
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text)

    print("\nTesto originale:", text)
    print("Token:", tokens)
    print("Token IDs:", token_ids)

    # Decodifica dei token
    decoded = tokenizer.decode(token_ids)
    print("Testo decodificato:", decoded)

example_1_basic_tokenization()

Dimensione del vocabolario GPT-2: 50257

Testo originale: Hello, how are you?
Token: ['Hello', ',', 'Ġhow', 'Ġare', 'Ġyou', '?']
Token IDs: [15496, 11, 703, 389, 345, 30]
Testo decodificato: Hello, how are you?


In [10]:
def example_2_token_prediction():
    """Esempio 2: Predizione del token successivo"""
    text = "Hello, how are"
    inputs = tokenizer.encode(text, return_tensors='pt')

    # Ottieni le predizioni del modello
    with torch.no_grad():
        outputs = model(inputs)
        predictions = outputs.logits[0, -1, :]

    # Prendi i 5 token più probabili
    probs = torch.softmax(predictions, dim=-1)
    top_5 = torch.topk(probs, 5)

    print(f"\nContesto: '{text}'")
    print("\nI 5 token più probabili come successivi sono:")
    for i, (token_id, prob) in enumerate(zip(top_5.indices, top_5.values)):
        token = tokenizer.decode([token_id])
        print(f"{i+1}. '{token}' con probabilità {prob:.3f}")

example_2_token_prediction()


Contesto: 'Hello, how are'

I 5 token più probabili come successivi sono:
1. ' you' con probabilità 0.663
2. ' we' con probabilità 0.131
3. ' the' con probabilità 0.028
4. ' your' con probabilità 0.025
5. ' they' con probabilità 0.024


In [11]:
def example_3_context_influence():
    """Esempio 3: Influenza del contesto sulla predizione"""
    contexts = [
        "The dog barks",
        "The cat mieows",
        "The student reads"
    ]

    print("\nInfluenza del contesto sulla predizione:")
    for context in contexts:
        inputs = tokenizer.encode(context, return_tensors='pt')
        with torch.no_grad():
            outputs = model(inputs)
            predictions = outputs.logits[0, -1, :]
            probs = torch.softmax(predictions, dim=-1)
            top_3 = torch.topk(probs, 3)

        print(f"\nContesto: '{context}'")
        print("Top 3 predizioni:")
        for i, (token_id, prob) in enumerate(zip(top_3.indices, top_3.values)):
            token = tokenizer.decode([token_id])
            print(f"  {i+1}. '{token}' (prob: {prob:.3f})")


example_3_context_influence()


Influenza del contesto sulla predizione:

Contesto: 'The dog barks'
Top 3 predizioni:
  1. ' at' (prob: 0.184)
  2. ' and' (prob: 0.164)
  3. ',' (prob: 0.112)

Contesto: 'The cat mieows'
Top 3 predizioni:
  1. ' the' (prob: 0.105)
  2. ' her' (prob: 0.044)
  3. ' and' (prob: 0.041)

Contesto: 'The student reads'
Top 3 predizioni:
  1. ' a' (prob: 0.275)
  2. ' the' (prob: 0.230)
  3. ' from' (prob: 0.038)


In [ ]:
def generate_text(prompt, model, tokenizer, max_length=100, num_return_sequences=1):
    """
    Genera testo a partire da un prompt

    Args:
        prompt (str): Il testo di input
        model: Il modello GPT-2
        tokenizer: Il tokenizer GPT-2
        max_length (int): Lunghezza massima dell'output
        num_return_sequences (int): Numero di sequenze da generare

    Returns:
        list: Lista di testi generati
    """
    # Codifichiamo il prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # Spostiamo gli input sul device corretto
    inputs = inputs.to(model.device)

    # Generiamo il testo
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodifichiamo e restituiamo il testo generato
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [18]:

def generate_text(prompt, model, tokenizer, max_length=100, num_return_sequences=1):
    """Genera testo a partire da un prompt"""
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    inputs = inputs.to(model.device)

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Esempio di generazione
prompt = "Artificial Intelligence in the future"

generated_texts = generate_text(prompt, model, tokenizer, max_length=150)
display(Markdown(f"{generated_texts[0]}"))


Artificial Intelligence in the future is a huge challenge for us. This technology will need to take some time to develop. In the meantime, we can see how other people's work will be used to improve the experience of their customers, and we will continue to work with developers to make sure the technology is used in their products and services.

For more information about the team at IBM, visit their website.